# Problemas de prueba de hipótesis

## Ejercicio 1

Usted es un nutricionista que investiga dos tipos diferentes de dietas para ver si existe una diferencia significativa en la pérdida de peso después de un mes. Eliges dos grupos aleatorios de personas; un grupo sigue la primera dieta y el otro sigue la segunda. Al final del mes, se registra la pérdida de peso (en kg) de cada persona:

| Diet 1 | Diet 2 |
|:-------|:-------|
| 2.0 | 3.0 |
| 2.5 | 3.2 |
| 3.0 | 3.1 |
| 2.8 | 2.9 |
| 2.3 | 2.8 |
| 2.7 | 3.0 |
| 2.5 | 3.2 |

Con estos datos se busca responder a la siguiente pregunta: ¿Existe una diferencia significativa en la pérdida de peso promedio entre las personas que siguieron la primera dieta y las que siguieron la segunda dieta?

Para sacar conclusiones, siga los siguientes puntos:

-Enunciar la hipótesis: hipótesis nula y alternativa.
-Realizar la prueba para comprobar la hipótesis. Puede utilizar la prueba t de Student.
-Analizar las conclusiones.

In [1]:
# TODO
import numpy as np
from scipy import stats

diet1 = np.array([2.0, 2.5, 3.0, 2.8, 2.3, 2.7, 2.5])
diet2 = np.array([3.0, 3.2, 3.1, 2.9, 2.8, 3.0, 3.2])

def welch_ttest(x, y, alpha=0.05):
    """
    Prueba t de Welch (independiente, varianzas no asumidas iguales).
    Devuelve un dict con estadísticos clave y una conclusión textual.
    """
    mean_x, mean_y = np.mean(x), np.mean(y)
    sd_x, sd_y = np.std(x, ddof=1), np.std(y, ddof=1)
    n_x, n_y = len(x), len(y)

    t_stat, p_value = stats.ttest_ind(x, y, equal_var=False)

    s_pooled = np.sqrt(((n_x-1)*sd_x**2 + (n_y-1)*sd_y**2) / (n_x + n_y - 2))
    cohen_d = (mean_y - mean_x) / s_pooled

    reject = p_value < alpha
    conclusion = (
        f"Con α={alpha:.2f}, p={p_value:.4f} -> "
        + ("Rechazamos H0: hay evidencia de diferencia significativa."
           if reject else
           "No rechazamos H0: no hay evidencia suficiente de diferencia.")
    )

    return {
        "mean_diet1": mean_x,
        "mean_diet2": mean_y,
        "sd_diet1": sd_x,
        "sd_diet2": sd_y,
        "n_diet1": n_x,
        "n_diet2": n_y,
        "t_stat": t_stat,
        "p_value": p_value,
        "cohen_d": cohen_d,
        "alpha": alpha,
        "conclusion": conclusion
    }

res = welch_ttest(diet1, diet2, alpha=0.05)

print("=== Prueba t de Welch: Dieta 1 vs Dieta 2 ===")
print(f"n1={res['n_diet1']}, n2={res['n_diet2']}")
print(f"Media Dieta 1 = {res['mean_diet1']:.3f} kg (sd={res['sd_diet1']:.3f})")
print(f"Media Dieta 2 = {res['mean_diet2']:.3f} kg (sd={res['sd_diet2']:.3f})")
print(f"t = {res['t_stat']:.3f}, p = {res['p_value']:.4f}")
print(f"Efecto (Cohen's d) = {res['cohen_d']:.3f}")
print(res["conclusion"])

=== Prueba t de Welch: Dieta 1 vs Dieta 2 ===
n1=7, n2=7
Media Dieta 1 = 2.543 kg (sd=0.331)
Media Dieta 2 = 3.029 kg (sd=0.150)
t = -3.538, p = 0.0071
Efecto (Cohen's d) = 1.891
Con α=0.05, p=0.0071 -> Rechazamos H0: hay evidencia de diferencia significativa.


## ANOVA

**ANOVA** (*Analysis of Variance - Análisis de variación*) Es una técnica estadística utilizada para comparar las medidas de dos o más grupos. La idea detrás de ANOVA es descomponer la variabilidad total de los datos en dos componentes: variabilidad entre grupos y variabilidad dentro de los grupos:

- **Variabilidad entre grupos**: Esta variabilidad se refiere a las diferencias entre las medias del grupo. Si esta variabilidad es considerablemente mayor que la variabilidad dentro del grupo, podría ser una indicación de que al menos una de las medias del grupo es diferente.
- **Variabilidad dentro del grupo**: Esta variabilidad se refiere a la dispersión de los datos dentro de cada grupo. Si todos los grupos tienen una variabilidad similar, entonces cualquier diferencia notable en las medias de los grupos podría considerarse significativa.

Las hipótesis en ANOVA suelen incluir:

- **Hipótesis nula** ($H₀$): Las medias de todos los grupos son iguales.
- **Hipótesis alternativa** ($H₁$): Al menos una de las medias del grupo es diferente.

Si el resultado de la prueba ANOVA es significativo (por ejemplo, un valor p inferior a un umbral como 0,05), esto sugiere que al menos la media de un grupo es diferente.

## Ejercicio 2

Un agricultor decide probar tres tipos diferentes de fertilizantes para determinar si alguno es superior en términos de producción de maíz. El agricultor planta maíz en 15 parcelas idénticas y utiliza los tres fertilizantes (5 parcelas para cada tipo). Al final de la temporada mide el rendimiento de maíz (en kg) de cada parcela, con el siguiente resultado:

| Fertilizante 1 | Fertilizante 2 | Fertilizante 3 |
|:-------------|:-------------|:-------------|
| 20 | 22 | 24 |
| 21 | 21 | 23 |
| 20 | 23 | 22 |
| 19 | 22 | 23 |
| 20 | 21 | 24 |

Con estos datos busca responder la siguiente pregunta: ¿Existe una diferencia significativa en el rendimiento promedio del maíz entre los tres tipos de fertilizantes?

Para ayudarte, sigue los siguientes puntos:

-Enunciar la hipótesis: hipótesis nula y alternativa.
-Realizar la prueba ANOVA.
-Analizar las conclusiones.
-Si un fertilizante es mejor que otro, ¿cómo podemos saberlo?

In [2]:
# TODO
import numpy as np
import pandas as pd
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd


fert1 = np.array([20, 21, 20, 19, 20])
fert2 = np.array([22, 21, 23, 22, 21])
fert3 = np.array([24, 23, 22, 23, 24])

df = pd.DataFrame({
    "rendimiento": np.concatenate([fert1, fert2, fert3]),
    "fertilizante": (["F1"]*len(fert1) + ["F2"]*len(fert2) + ["F3"]*len(fert3))
})

modelo = ols('rendimiento ~ C(fertilizante)', data=df).fit()
anova_tbl = sm.stats.anova_lm(modelo, typ=2) 

print("=== ANOVA una vía (SC tipo II) ===")
print(anova_tbl, "\n")

ss_between = anova_tbl.loc['C(fertilizante)', 'sum_sq']
ss_within  = anova_tbl.loc['Residual', 'sum_sq']
df_between = int(anova_tbl.loc['C(fertilizante)', 'df'])
df_within  = int(anova_tbl.loc['Residual', 'df'])
ss_total   = ss_between + ss_within

eta_sq   = ss_between / ss_total
omega_sq = (ss_between - df_between * (ss_within / df_within)) / (ss_total + (ss_within / df_within))

print(f"η² (eta squared)  = {eta_sq:.4f}")
print(f"ω² (omega squared)= {omega_sq:.4f}\n")

residuos = modelo.resid
sh_w, sh_p = stats.shapiro(residuos)

print("=== Supuestos ===")
print(f"Shapiro-Wilk residuos: W={sh_w:.3f}, p={sh_p:.4f}  -> ",
      "No evidencia contra normalidad" if sh_p>=0.05 else "Posible no-normalidad")

lev_w, lev_p = stats.levene(fert1, fert2, fert3, center='median')
print(f"Levene (varianzas iguales): W={lev_w:.3f}, p={lev_p:.4f}  -> ",
      "Varianzas homogéneas" if lev_p>=0.05 else "Varianzas desiguales", "\n")

p_anova = anova_tbl.loc['C(fertilizante)', 'PR(>F)']

if p_anova < 0.05:
    print("Conclusión ANOVA: p<0.05 -> Rechazamos H0. Al menos una media difiere.\n")
    
    if lev_p >= 0.05:
        print("=== Comparaciones múltiples (Tukey HSD) ===")
        tukey = pairwise_tukeyhsd(endog=df["rendimiento"],
                                  groups=df["fertilizante"],
                                  alpha=0.05)
        print(tukey.summary())
    else:
        print("Las varianzas parecen desiguales (Levene p<0.05).")
        print("Sugerencia: usar un post hoc robusto como Games-Howell (scikit-posthocs) o Welch-ANOVA.")
     
else:
    print("Conclusión ANOVA: p>=0.05 -> No rechazamos H0. No hay evidencia suficiente de diferencias.\n")
    print("No es necesario aplicar comparaciones múltiples.")

=== ANOVA una vía (SC tipo II) ===
                    sum_sq    df          F   PR(>F)
C(fertilizante)  25.733333   2.0  20.315789  0.00014
Residual          7.600000  12.0        NaN      NaN 

η² (eta squared)  = 0.7720
ω² (omega squared)= 0.7203

=== Supuestos ===
Shapiro-Wilk residuos: W=0.949, p=0.5148  ->  No evidencia contra normalidad
Levene (varianzas iguales): W=0.222, p=0.8040  ->  Varianzas homogéneas 

Conclusión ANOVA: p<0.05 -> Rechazamos H0. Al menos una media difiere.

=== Comparaciones múltiples (Tukey HSD) ===
Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj  lower  upper  reject
--------------------------------------------------
    F1     F2      1.8 0.0099 0.4572 3.1428   True
    F1     F3      3.2 0.0001 1.8572 4.5428   True
    F2     F3      1.4 0.0409 0.0572 2.7428   True
--------------------------------------------------
